In [1]:
# 필수(?) 라이브러리 호출
import pandas as pd
import numpy as np

In [2]:
# 경고가 많아지면 곤란해서 추가
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 일단 분석할 데이터를 가져오는데
df = pd.read_csv("../raw_data/전체데이터_2011_코스피코스닥_v1.2.csv", thousands=',', encoding="CP949").reset_index(drop=True)
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS),이자보상배율(이자비용)(IFRS연결)
0,(주)CMG제약,58820,11-Dec,3.94,3.53
1,(주)CMG제약,58820,12-Dec,-7.45,-7.75
2,(주)CMG제약,58820,13-Dec,0.73,0.70
3,(주)CMG제약,58820,14-Dec,3.63,3.60
4,(주)CMG제약,58820,15-Dec,-32.20,-32.22
...,...,...,...,...,...
15145,흥아해운(주),3280,15-Dec,0.79,1.02
15146,흥아해운(주),3280,16-Dec,0.04,0.21
15147,흥아해운(주),3280,17-Dec,-0.61,-0.42
15148,흥아해운(주),3280,18-Dec,-1.26,-1.16


In [4]:
# df.drop(columns=['Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8'], inplace=True)

In [5]:
df['이자보상배율(이자비용)(IFRS)'].value_counts()

 1.000000e+09    223
 1.560000e+00     18
 1.700000e-01     18
 0.000000e+00     17
 1.010000e+00     17
                ... 
-1.902062e+04      1
-1.618000e+01      1
 2.090200e+03      1
-1.841000e+01      1
 2.520200e+02      1
Name: 이자보상배율(이자비용)(IFRS), Length: 7587, dtype: int64

In [6]:
df['이자보상배율(이자비용)(IFRS연결)'].value_counts()

 0.00      19
 0.62      16
 1.30      15
 1.13      15
 2.08      15
           ..
 15.79      1
 40.16      1
 45.14      1
-6.01       1
 252.02     1
Name: 이자보상배율(이자비용)(IFRS연결), Length: 7556, dtype: int64

In [7]:
# 분석 과정에서 NaN이 NaN이 아닌 다른 어떤 값인 것으로 판명되었습니다.
# 어쩔 수 없이 분석의 용이함을 위해 임의의 값을 채웠습니다
#df = df.fillna(77777777.77)

# 9999도 결측치니까 대체를 해줍니다.
#df.replace('999999999', 77777777.77, inplace=True)
#df.replace('999999999', 11111111.11, inplace=True)


# 이건 우리의 기준에 맞는 이자보상 배율을 저장하기 위한 컬럼입니다.
df['이자보상배율'] = 0.0

In [8]:
# # csv 한바퀴 돌립니다
# for i in range(0,len(df)-1):
#     #print(i)

#     # 구 기준 회계값에 결측치가 없다면
#     if df.iloc[i]['이자보상배율(이자비용)(IFRS)'] != 77777777.77:
#         # 신 기준 회계값에도 결측치가 없다면
#         if df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'] != 77777777.77:
#             # 신 회계값 삽입
#             df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
#             #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

#         # 신 회계가 결측치라면
#         else:
#             df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS)'] # 구 회계값 삽입
#             #print(df.iloc[i]['이자보상배율(이자비용)'])

#     # 구 기준 회계값에 결측치가 있다면
#     else:
#         # 신 회계 기준 회계값에 결측치가 없다면
#         if df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'] != 77777777.77:
#             # 신 회계값 삽입
#             df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
#             #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

#         # 이자보상배율에 대한 데이터가 아예 없는 경우 임의의 값 삽입
#         else:
#             df.loc[i, "이자보상배율"] = 11111111.11

In [9]:
# csv 한바퀴 돌립니다
for i in range(0,len(df)):
    #print(i)

    # 개별 재무제표에 결측치가 없다면
    if not pd.isna(df.iloc[i]['이자보상배율(이자비용)(IFRS)']):
        # 연결 재무제표에도 결측치가 없다면
        if not pd.isna(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']):
            # 연결 재무제표 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
            #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

        # 연결 재무제표가 결측치라면
        else:
            # 개별 재무제표 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS)']
            #print(df.iloc[i]['이자보상배율(이자비용)'])

    # 개별 재무제표에 결측치가 있다면
    else:
        # 연결 재무제표에 결측치가 없다면
        if not pd.isna(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']):
            # 연결 재무제표 삽입
            df.loc[i, "이자보상배율"] = df.iloc[i]['이자보상배율(이자비용)(IFRS연결)']
            #print(df.iloc[i]['이자보상배율(이자비용)(IFRS연결)'])

        # 둘 다 결측치가 존재하는 경우 통과
        else:
            pass

In [10]:
df

,회사명,거래소코드,회계년도,이자보상배율(이자비용)(IFRS),이자보상배율(이자비용)(IFRS연결),이자보상배율
0,(주)CMG제약,58820,11-Dec,3.94,3.53,3.53
1,(주)CMG제약,58820,12-Dec,-7.45,-7.75,-7.75
2,(주)CMG제약,58820,13-Dec,0.73,0.70,0.70
3,(주)CMG제약,58820,14-Dec,3.63,3.60,3.60
4,(주)CMG제약,58820,15-Dec,-32.20,-32.22,-32.22
...,...,...,...,...,...,...
15145,흥아해운(주),3280,15-Dec,0.79,1.02,1.02
15146,흥아해운(주),3280,16-Dec,0.04,0.21,0.21
15147,흥아해운(주),3280,17-Dec,-0.61,-0.42,-0.42
15148,흥아해운(주),3280,18-Dec,-1.26,-1.16,-1.16


In [11]:
df['이자보상배율'].value_counts()

0.000000e+00    942
1.000000e+09    137
6.200000e-01     17
1.130000e+00     15
2.080000e+00     15
               ... 
1.962000e+02      1
1.412600e+02      1
2.491500e+02      1
5.443300e+02      1
2.520200e+02      1
Name: 이자보상배율, Length: 8250, dtype: int64

In [12]:
# df.to_csv("../raw_data/전체데이터_2011_코스피코스닥_1차전처리_v1.0.csv", encoding="CP949")

In [13]:
# 우리가 앞으로 작업을 엄청나게 복잡하게 할 걸 생각하면
# 중간중간 결과들을 데이터프레임으로 만들고 빼주는 게 좋습니다
df_2 = df[['회사명', '거래소코드', '회계년도', '이자보상배율']]
df_2

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,11-Dec,3.53
1,(주)CMG제약,58820,12-Dec,-7.75
2,(주)CMG제약,58820,13-Dec,0.70
3,(주)CMG제약,58820,14-Dec,3.60
4,(주)CMG제약,58820,15-Dec,-32.22
...,...,...,...,...
15145,흥아해운(주),3280,15-Dec,1.02
15146,흥아해운(주),3280,16-Dec,0.21
15147,흥아해운(주),3280,17-Dec,-0.42
15148,흥아해운(주),3280,18-Dec,-1.16


In [14]:
# # 일단 우리의 소중한 결과물을 빼놓도록 하죠
# # 버전 올리는 거 잊지 마세요

# df_2.to_csv("../csv_data/이자보상배율_v1.0.csv", encoding="CP949")

In [15]:
df_2

,회사명,거래소코드,회계년도,이자보상배율
0,(주)CMG제약,58820,11-Dec,3.53
1,(주)CMG제약,58820,12-Dec,-7.75
2,(주)CMG제약,58820,13-Dec,0.70
3,(주)CMG제약,58820,14-Dec,3.60
4,(주)CMG제약,58820,15-Dec,-32.22
...,...,...,...,...
15145,흥아해운(주),3280,15-Dec,1.02
15146,흥아해운(주),3280,16-Dec,0.21
15147,흥아해운(주),3280,17-Dec,-0.42
15148,흥아해운(주),3280,18-Dec,-1.16


In [16]:
df_2['회계년도'] = pd.to_datetime(df_2['회계년도'].str.upper(), format="%d-%b", yearfirst=False).dt.strftime("20%d")

In [79]:
f_test = []
count = 0
company = df_2.iloc[0]['회사명']

for i in range(0, len(df_2)):

    # 회사명 다르면
    if company != df_2.iloc[i]['회사명']:
        # 회사명 기록하고
        company = df_2.iloc[i]['회사명']
        # 카운트 초기화하고
        count = 0

    # 2년 연속 걸릴 때
    if count == 2 :
        # 카운트 초기화하고
        #count = 0
        continue

    # 기록은 기록대로 다시 하고
    if df_2.iloc[i]['이자보상배율'] < 1:
        count += 1
    else:
        count = 0

    f_test.append(df_2.loc[i])
    print(i, count)


0 0
1 1
2 2
9 1
10 2
18 1
19 0
20 1
21 0
22 1
23 0
24 0
25 1
26 2
27 1
28 2
36 1
37 0
38 1
39 0
40 0
41 0
42 0
43 1
44 2
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 1
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 1
64 0
65 0
66 0
67 0
68 1
69 2
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 1
83 2
90 0
91 1
92 2
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 1
109 2
113 0
114 0
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 1
128 2
131 0
132 0
133 0
134 1
135 0
136 1
137 2
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 1
166 0
167 0
168 0
169 0
170 0
171 1
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
180 0
181 0
182 0
183 0
184 0
185 0
186 0
187 0
188 0
189 0
190 0
191 0
192 0
193 1
194 2
199 0
200 0
201 0
202 0
203 0
204 0
205 0
206 0
207 0
208 0
209 1
210 2
217 0
218 0
219 1
220 2
226 1
227 0
228 0
229 0
230 0
231 0
232 0
233 0
23

In [80]:
#pd.DataFrame(f_test)
pd.DataFrame(f_test).to_csv("checking.csv", encoding="CP949")

In [273]:
# 현재 데이터 기준으로 한계기업과 4년 연속 이.보.배 1 미만인 기업을 찾을 겁니다

f_company = [] # 한계기업을 저장하는 리스트
f_company_x = [] # 4년 연속 이자보상배율 1 미만인 기업 삽입
f_year = [] # 3년째 최초 연도
f_year_x = [] # 4년째 최초 연도
count = 0 # 한계기업 카운트용
company = 'x' # 한계기업 기록용

# 이거도 csv 한바퀴 돌립니다
for i in range(0, len(df_2)):

    # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
    if company == df.iloc[i]['회사명']:
        # 다만 그 기업이 한계기업인데
        if count > 3:
            # 4년 연속이네요? 기록!!!
            if df_2.iloc[i]['이자보상배율'] < 1:
                f_company_x.append(company)
                f_year_x.append(df.iloc[i]['회계년도'])
                count = 0

                # 반복문에서 볼일 다 끝났으니 통과합니다
                continue

        # 그게 아니라면 반복문을 통과합니다
        continue

    # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
    if df_2.iloc[i]['이자보상배율'] < 1:
            count += 1
    # 도중에 이자보상배율이 1 미만이면 탈출합니다
    else:
        count = 0

    # 한계 기업 클럽에 오셨습니다. 환영합니다.
    if count == 3:
        company = df_2.iloc[i]['회사명']
        f_company.append(company)
        f_year.append(df.iloc[i]['회계년도'])
        count += 1

In [274]:
fda = pd.DataFrame(f_year)
fdb = pd.DataFrame(f_year_x)
fdc = pd.DataFrame(f_company)
fdd = pd.DataFrame(f_company_x)
fde = pd.concat([fda, fdc, fdb, fdd], axis=1)
# fde.to_csv("../csv_data/나락시험.csv", encoding="CP949")

In [275]:
fde

,0,0,0,0
0,13-Dec,(주)DB하이텍,14-Dec,(주)DB하이텍
1,11-Dec,(주)HJ중공업,12-Dec,(주)HJ중공업
2,14-Dec,(주)ST,15-Dec,(주)ST
3,14-Dec,(주)TCC스틸,18-Dec,(주)강스템바이오텍
4,17-Dec,(주)강스템바이오텍,19-Dec,(주)강원에너지
...,...,...,...,...
530,13-Dec,현대바이오사이언스(주),NaN,NaN
531,14-Dec,현우산업(주),NaN,NaN
532,14-Dec,휴림로봇(주),NaN,NaN
533,16-Dec,휴마시스(주),NaN,NaN


In [276]:
# 분명히 이거보다 더 좋은 방법이 있을건데
# 맷돌이 굴러가질 않네요

# 이건 f_company 리스트에 들어잇는 데이터와 일치하면 True, 아니면 False를 반환합니다
df_4 = df_2.isin(f_company)
df_4

,회사명,거래소코드,회계년도,이자보상배율
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
15145,True,False,False,False
15146,True,False,False,False
15147,True,False,False,False
15148,True,False,False,False


In [277]:
df_5 = [] # 임시 저장용 리스트

# 그렇게 만든 데이터프레임을 기존과 비교해서 True인 애들만 뽑아옵니다
for i in range(0, len(df_4)-1):
    if df_4.iloc[i]["회사명"] == True:
        df_5.append(df_2.loc[i])

In [278]:
# 그리고 저장을 합니다
df_5_frame = pd.DataFrame(df_5).reset_index(drop=True)
# df_5_frame.to_csv("../csv_data/한계기업후보v2.csv", encoding="CP949")

In [279]:
# 4년 연속 이자 보상 배율이 1 미만인 기업들도 똑같이 해봅니다

df_6 = df_2.isin(f_company_x)
df_7 = []

for i in range(0, len(df_6)-1):
    if df_6.iloc[i]["회사명"] == True:
        df_7.append(df_2.loc[i])

df_7_frame = pd.DataFrame(df_7).reset_index(drop=True)

In [280]:
# # 중간중간 저장은 선택이 아닌 필수
# df_7_frame.to_csv("../csv_data/회생불가1차_v1.csv", encoding="CP949")

In [281]:
# 그래서 4년 연속인 애들이 얼마나 될까요? 밑에 숫자를 봅시다
df_7_frame.copy().drop_duplicates(['회사명'])

,회사명,거래소코드,회계년도,이자보상배율
0,(주)DB하이텍,990,2011,-0.75
9,(주)HJ중공업,97230,2011,0.55
18,(주)ST,11810,2011,1.66
27,(주)강스템바이오텍,217730,2015,-5.41
32,(주)강원에너지,114190,2011,393.90
...,...,...,...,...
3428,한일철강(주),2220,2011,0.87
3437,현대바이오사이언스(주),48410,2011,-18.41
3446,휴림로봇(주),90710,2011,2.96
3455,휴마시스(주),205470,2014,-0.76


In [282]:
# 한계기업 후보 기업의 숫자는 밑에 카운트를 봅시다
df_5_frame.copy().drop_duplicates(['회사명'])

,회사명,거래소코드,회계년도,이자보상배율
0,(주)DB하이텍,990,2011,-0.75
9,(주)HJ중공업,97230,2011,0.55
18,(주)ST,11810,2011,1.66
27,(주)TCC스틸,2710,2011,1.41
36,(주)강스템바이오텍,217730,2015,-5.41
...,...,...,...,...
4435,현대바이오사이언스(주),48410,2011,-18.41
4444,현우산업(주),92300,2011,2.86
4453,휴림로봇(주),90710,2011,2.96
4462,휴마시스(주),205470,2014,-0.76


In [283]:
df_test = df_5_frame.copy()
df_test_2 = df_7_frame.copy()

In [284]:
df_test_t = df_test[df_test['이자보상배율']==11111111.11].index
df_test.drop(df_test_t, inplace=True)

In [285]:
df_test_t2 = df_test_2[df_test_2['이자보상배율']==11111111.11].index
df_test_2.drop(df_test_t2, inplace=True)

In [286]:
# df_test.to_csv("../csv_data/test_1_한계.csv", encoding="CP949")
# df_test_2.to_csv("../csv_data/test_2_부실.csv", encoding="CP949")

In [287]:
df_test

,회사명,거래소코드,회계년도,이자보상배율
0,(주)DB하이텍,990,2011,-0.75
1,(주)DB하이텍,990,2012,-0.31
2,(주)DB하이텍,990,2013,-0.16
3,(주)DB하이텍,990,2014,0.67
4,(주)DB하이텍,990,2015,2.50
...,...,...,...,...
4471,흥아해운(주),3280,2014,1.12
4472,흥아해운(주),3280,2015,1.02
4473,흥아해운(주),3280,2016,0.21
4474,흥아해운(주),3280,2017,-0.42


In [288]:
# 한계기업 후보 리스트가 어쨌든 메인이고
# 그 안에서 우리는 부실기업을 찾자는 것이고
df_5_frame['부실기업'] = 0
df_9 = df_5_frame.isin(df_7_frame)
for i in range(0, len(df_5_frame)):
    if df_9.iloc[i]['회사명'] == True:
        df_5_frame.loc[i, '부실기업'] = 1
df_5_frame

,회사명,거래소코드,회계년도,이자보상배율,부실기업
0,(주)DB하이텍,990,2011,-0.75,1
1,(주)DB하이텍,990,2012,-0.31,1
2,(주)DB하이텍,990,2013,-0.16,1
3,(주)DB하이텍,990,2014,0.67,1
4,(주)DB하이텍,990,2015,2.50,1
...,...,...,...,...,...
4471,흥아해운(주),3280,2014,1.12,0
4472,흥아해운(주),3280,2015,1.02,0
4473,흥아해운(주),3280,2016,0.21,0
4474,흥아해운(주),3280,2017,-0.42,0


In [289]:
# df_test_alpha = pd.read_csv("../raw_data/00_기계비율_개별.csv", encoding="CP949")
# df_test_beta = pd.read_csv("../raw_data/00_기계비율_연결.csv", encoding="CP949")

In [290]:
# df_test_alpha.head(1)

In [291]:
# df_test_beta.head(1)

In [292]:
# df_test_alpha['회계년도'] = pd.to_datetime(df_test_alpha['회계년도'].str.upper(), format="%b-%d", yearfirst=False).dt.strftime("20%d")
# df_test_beta['회계년도'] = pd.to_datetime(df_test_beta['회계년도'].str.upper(), format="%b-%d", yearfirst=False).dt.strftime("20%d")

In [293]:
# datax = df_test_alpha['회사명'].value_counts()
# datax.to_csv('000_test.csv', header=None, encoding="CP949")

In [294]:
# datay = df_2['회사명'].value_counts()
# datay.to_csv('000_test1.csv', header=None, encoding="CP949")

In [295]:
# df_test_alpha.isin(df_2['회계년도'])

In [296]:
# f_machine_a = []
# f_machine_b = []

# for i in range(0, len(df_2)):
    
#     # 회사명이 이미 기록된 상태에서 또 중복으로 적을 필요는 없습니다
#     if company == df.iloc[i]['회사명']:
#         # 다만 그 기업이 한계기업인데
#         if count > 3:
#             # 어머? 4년 연속이네요? 기록!!!
#             if df_2.iloc[i]['이자보상배율'] < 1:
#                 f_company_x.append(company)
#                 f_year_x.append(df.iloc[i]['회계년도'])
#                 count = 0

#                 # 반복문에서 볼일 다 끝났으니 통과합니다
#                 continue

#         # 그게 아니라면 반복문을 통과합니다
#         continue

#     # 이자보상배율이 1 미만인 경우 카운트를 늘립니다
#     if df_2.iloc[i]['이자보상배율'] < 1:
#             count += 1
#     # 도중에 이자보상배율이 1 미만이면 탈출합니다
#     else:
#         count = 0

#     # 한계 기업 클럽에 오셨습니다. 환영합니다.
#     if count == 3:
#         company = df_2.iloc[i]['회사명']
#         f_company.append(company)
#         f_year.append(df.iloc[i]['회계년도'])
#         count += 1

In [297]:
df_rate = pd.read_csv("../raw_data/01_매출원가율감소여부2.csv", encoding="CP949")
df_rate.drop(columns=['Unnamed: 0'], inplace=True)

In [298]:
df_rate.head(1)

,구분,코드,산업명,2011년,2011년.mid,2012년,2012년.mid,2012년 감소율,2013년,2013년.mid,...,2016년 감소율,2017년,2017년.mid,2017년 감소율,2018년,2018년.mid,2018년 감소율,2019년,2019년.mid,2019년 감소율
0,(주)CMG제약,58820,의료용 물질 및 의약품 제조업,58.17,41.73,71.66,41.84,-23.19,61.54,43.25,...,6.43,56.57,52.32,5.15,54.4,54.69,3.84,50.66,57.6,6.88


In [299]:
df_rate['2012년비교'] = 0
df_rate['2013년비교'] = 0
df_rate['2014년비교'] = 0
df_rate['2015년비교'] = 0
df_rate['2016년비교'] = 0
df_rate['2017년비교'] = 0
df_rate['2018년비교'] = 0
df_rate['2019년비교'] = 0

In [300]:
df_rate.head(1)

,구분,코드,산업명,2011년,2011년.mid,2012년,2012년.mid,2012년 감소율,2013년,2013년.mid,...,2019년.mid,2019년 감소율,2012년비교,2013년비교,2014년비교,2015년비교,2016년비교,2017년비교,2018년비교,2019년비교
0,(주)CMG제약,58820,의료용 물질 및 의약품 제조업,58.17,41.73,71.66,41.84,-23.19,61.54,43.25,...,57.6,6.88,0,0,0,0,0,0,0,0


In [301]:
# # 당기 = 2012년
# # 전년도 매출원가비율(2011년)이 중간값(2011년 mid)보다 높고 = A
# # AND
# # [ 	당기에 해당비율이 4분위 아래로 떨어졌을 경우 = {2012년 - 2011년}/2012년 < -20
# # 	OR
# # 	차기에 해당비율이 4분위 아래로 떨어졌을 경우 = {2013년 - 2012년}/2013년 < -20 ] = B

# #year = 2012

# df_rate = df_rate.fillna(0)

# for i in range(0, 1):
#     # 2012년 ~ 2018년 순회공연
#     for j in range(5, len(df_rate.columns)-13, 4):
#         print(i,j)
#         if df_rate.iloc[i][j-1] > df_rate.iloc[i][j] :
#             if ((df_rate.iloc[i][j] - df_rate.iloc[i][j-3]) / df_rate.iloc[i][j]) < -20:
#                 df_rate.iloc[i, j+24] = 1
#             elif ((df_rate.iloc[i][j+3] - df_rate.iloc[i][j]) / df_rate.iloc[i][j]) < -20:
#                 df_rate.iloc[i, j+24] = 1

#     # 2019년 순회공연
#     if df_rate.iloc[i][23] > df_rate.iloc[i][24]:
#         if ((df_rate.iloc[i][26] - df_rate.iloc[i][23]) / df_rate.iloc[i][26]) < -20:
#             df_rate.iloc[i, 36]
#     else:
#         pass

In [302]:
year=2013
df_rate[str(year-1)+"년"]

0       71.66
1        0.00
2       73.71
3       67.22
4        0.00
        ...  
2018     0.00
2019     0.00
2020     0.00
2021    89.82
2022    33.51
Name: 2012년, Length: 2023, dtype: float64

In [303]:
# 당기 = 2012년
# 전년도 매출원가비율(2011년)이 중간값(2011년 mid)보다 높고 = A
# AND
# [ 	당기에 해당비율이 4분위 아래로 떨어졌을 경우 = {2012년 - 2011년}/2012년 < -20
# 	OR
# 	차기에 해당비율이 4분위 아래로 떨어졌을 경우 = {2013년 - 2012년}/2013년 < -20 ] = B

#year = 2012

for i in range(0, len(df_rate)):
    for year in range(2012, 2020):
        #print(year)
        if df_rate.iloc[i][str(year-1)+"년"] > df_rate.iloc[i][str(year-1)+"년.mid"]:
            if year != 2019:
                if ((df_rate.iloc[i][str(year)+"년"] - df_rate.iloc[i][str(year-1)+"년"]) / df_rate.iloc[i][str(year)+"년"]).round(2) < -20:
                    df_rate.loc[i, str(year)+"년비교"] = 1
                elif ((df_rate.iloc[i][str(year+1)+"년"] - df_rate.iloc[i][str(year)+"년"]) / df_rate.iloc[i][str(year+1)+"년"]).round(2) < -20:
                    df_rate.loc[i, str(year)+"년비교"] = 1
            else:
                if ((df_rate.iloc[i][str(year)+"년"] - df_rate.iloc[i][str(year-1)+"년"]) / df_rate.iloc[i][str(year)+"년"]).round(2) < -20:
                    df_rate.loc[i, str(year)+"년비교"] = 1

In [304]:
df_rate["2019년비교"].value_counts()

0    2016
1       7
Name: 2019년비교, dtype: int64

In [316]:
df_rate.head(1)

,구분,코드,산업명,2011년,2011년.mid,2012년,2012년.mid,2012년 감소율,2013년,2013년.mid,...,2019년.mid,2019년 감소율,2012년비교,2013년비교,2014년비교,2015년비교,2016년비교,2017년비교,2018년비교,2019년비교
0,(주)CMG제약,58820,의료용 물질 및 의약품 제조업,58.17,41.73,71.66,41.84,-23.19,61.54,43.25,...,57.6,6.88,0,0,0,0,0,0,0,0


In [305]:
df_m_a = pd.read_csv("../raw_data/00_기계비율_개별.csv", thousands=",", encoding="CP949")
df_m_b = pd.read_csv("../raw_data/00_기계비율_연결.csv", thousands=",", encoding="CP949")

In [306]:
df_m_a = df_m_a.iloc[:, 13:]

In [307]:
df_m_b = df_m_b.iloc[:, 13:]

In [308]:
df_m_c = df_m_a.copy()

In [309]:
df_m_a = df_m_a.replace("#DIV/0!", )
df_m_b = df_m_b.replace("#DIV/0!", )

In [310]:
pd.isna(df_m_b.iloc[0][3])

True

In [311]:
df_m_b.iloc[0][3] == pd.isna(df_m_c.iloc[0][3])

False

In [312]:
for i in range(0,len(df_m_c)):
    # 컬럼들 순회합니다
    for j in range(0, len(df_m_c.columns)):
        # df_m_a 에 결측치가 없다면
        if not pd.isna(df_m_a.iloc[i][j]):
            # df_m_b 에도 결측치가 없다면
            if not pd.isna(df_m_b.iloc[i][j]):
                # df_m_b 삽입
                df_m_c.iloc[i, j] = df_m_b.iloc[i][j]

            # df_m_b 가 결측치라면
            else:
                #df_m_a 값 삽입
                df_m_c.iloc[i, j] = df_m_a.iloc[i][j]

        # df_m_a 에 결측치가 있다면
        else:
            # df_m_b 에 결측치가 없다면
            if not pd.isna(df_m_b.iloc[i][j]):
                # df_m_b 삽입
                df_m_c.iloc[i, j] = df_m_b.iloc[i][j]

            # 둘 다 없으면 결측치로 넣어벼려요
            # 근데 아마 이거 동작 안 할 겁니다. 이미 채워넣었기 때문에...
            else:
                df_m_c.iloc[i, j] = 11111111.11

In [317]:
df_m_c.to_csv("../csv_data/00_기계_종합.csv", encoding="CP949")